In [1]:
import os
os.chdir("../")

In [3]:
import pandas as pd

In [4]:
data=pd.read_csv("artifacts/data_ingestion/train.csv")

In [20]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from src.SongPopularityPrediction.constants import *
from src.SongPopularityPrediction.utils.common import read_yaml, create_directories

In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [9]:
import os
from src.SongPopularityPrediction import logger
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.model_selection import train_test_split

In [10]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [33]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def transformation(self,X):
        y=X.pop('song_popularity')
        imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
        imputer = imputer.fit(X)
        X = imputer.transform(X)

        columns=['id', 'song_duration_ms', 'acousticness', 'danceability', 'energy',
                'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
                'speechiness', 'tempo', 'time_signature', 'audio_valence']
        X=pd.DataFrame(X,columns=columns)



        encoder = OneHotEncoder(handle_unknown='ignore')
        encoder_X = pd.DataFrame(encoder.fit_transform(X[['key','time_signature']]).toarray())
        X=X.join(encoder_X)
        X.drop(['key','time_signature'],axis=1,inplace=True)

        x =X.values #returns a numpy array
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        X_scaled = pd.DataFrame(x_scaled)
        X_scaled['song_popularity']=y
        return X_scaled
    
    def train_test_splitting(self):
        print(self.config.root_dir)
        print(self.config.data_path)
        data = pd.read_csv(self.config.data_path)

        data=self.transformation(data)
        train,test=train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"),index=False)

        logger.info("Data tranformation and splitting it into traina and test")
        logger.info(train.shape)
        logger.info(test.shape)

In [38]:
try:
    config=ConfigurationManager()
    data_transformation_config=config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2023-12-24 01:39:08,533: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-24 01:39:08,539: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-24 01:39:08,553: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-24 01:39:08,556: INFO: common: created directory at: artifacts]


[2023-12-24 01:39:08,658: INFO: common: created directory at: artifacts/data_transformation]
artifacts/data_transformation
artifacts/data_ingestion/train.csv
[2023-12-24 01:39:10,127: INFO: 2405595147: Data tranformation and splitting it into traina and test]
[2023-12-24 01:39:10,127: INFO: 2405595147: (30000, 30)]
[2023-12-24 01:39:10,127: INFO: 2405595147: (10000, 30)]
